<a href="https://colab.research.google.com/github/Pratsa09/Dphi_flask/blob/main/RF_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/dphi-official/Datasets/master/Loan_Data/loan_train.csv" )
df.head()

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,LP002305,Female,No,0,Graduate,No,4547,0.0,115.0,360.0,1.0,Semiurban,1
1,1,LP001715,Male,Yes,3+,Not Graduate,Yes,5703,0.0,130.0,360.0,1.0,Rural,1
2,2,LP002086,Female,Yes,0,Graduate,No,4333,2451.0,110.0,360.0,1.0,Urban,0
3,3,LP001136,Male,Yes,0,Not Graduate,Yes,4695,0.0,96.0,NaN,1.0,Urban,1
4,4,LP002529,Male,Yes,2,Graduate,No,6700,1750.0,230.0,300.0,1.0,Semiurban,1


In [ ]:
df.shape

(491, 14)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491 entries, 0 to 490
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         491 non-null    int64  
 1   Loan_ID            491 non-null    object 
 2   Gender             481 non-null    object 
 3   Married            490 non-null    object 
 4   Dependents         482 non-null    object 
 5   Education          491 non-null    object 
 6   Self_Employed      462 non-null    object 
 7   ApplicantIncome    491 non-null    int64  
 8   CoapplicantIncome  491 non-null    float64
 9   LoanAmount         475 non-null    float64
 10  Loan_Amount_Term   478 non-null    float64
 11  Credit_History     448 non-null    float64
 12  Property_Area      491 non-null    object 
 13  Loan_Status        491 non-null    int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 53.8+ KB


In [ ]:
df['Gender'] = df['Gender'].fillna('Not Specified')
df['Self_Employed'] = df['Self_Employed'].fillna('Not Specified')
df['Married'] = df['Married'].fillna('Not Specified')

In [ ]:
m1 = df['LoanAmount'].mean()
df['LoanAmount'] = df['LoanAmount'].fillna(m1)

In [ ]:
m2 = df['Loan_Amount_Term'].mean()
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].fillna(m2)

In [ ]:
m3 = df['Credit_History'].median()
df['Credit_History'] = df['Credit_History'].fillna(m3)

In [ ]:
df = df.replace('3+', '3')

In [ ]:
m4 = df['Dependents'].median()
df['Dependents'] = df['Dependents'].fillna(m4)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
df['Gender'] = le.fit_transform(df['Gender'])

In [ ]:
df['Married'] = le.fit_transform(df['Married'])

In [ ]:
df['Education'] = le.fit_transform(df['Education'])

In [ ]:
df['Property_Area'] = le.fit_transform(df['Property_Area'])

In [ ]:
df['Self_Employed'] = le.fit_transform(df['Self_Employed'])

In [ ]:
df = df.drop('Loan_ID',axis = 1)

In [ ]:
df['Dependents'] = df['Dependents'].astype('int64')

In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,0,0,0,0,4547,0.0,115.0,360.000000,1.0,1,1
1,1,2,3,1,2,5703,0.0,130.0,360.000000,1.0,0,1
2,0,2,0,0,0,4333,2451.0,110.0,360.000000,1.0,2,0
3,1,2,0,1,2,4695,0.0,96.0,341.297071,1.0,2,1
4,1,2,2,0,0,6700,1750.0,230.0,300.000000,1.0,1,1


In [ ]:
df.dtypes

Gender                 int64
Married                int64
Dependents             int64
Education              int64
Self_Employed          int64
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area          int64
Loan_Status            int64
dtype: object

In [ ]:
# data preprocessing of the training dataset complete

In [ ]:
X = df.drop(['Loan_Status'],axis = 1)
y = df['Loan_Status']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.25 , random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 80 ,criterion = 'gini', random_state = 42)
rf_model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=80,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
rf_model.score(X_val,y_val)

0.8211382113821138

In [ ]:
# Saving and pickling the model
import pickle

In [ ]:
pickle_out = open("rf_model.pkl", mode = "wb") 
pickle.dump(rf_model, pickle_out) 
pickle_out.close()

In [ ]:
# moving onto deployment

In [ ]:
!pip install pyngrok

In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import pickle
import streamlit as st
 
# loading the trained model
pickle_in = open('rf_model.pkl', 'rb') 
rf_classifier = pickle.load(pickle_in)
 
@st.cache()
  
# defining the function which will make the prediction using the data which the user inputs 
def prediction(Gender, Married, ApplicantIncome, LoanAmount, Loan_Amount_Term):   
 
    # Pre-processing user input    
    if Gender == "Male":
        Gender = 0
    elif Gender == "Female":
      Gender = 1
    else:
      Gender = 2
 
    if Married == "Unmarried":
        Married = 0
    elif Married =="Married":
        Married = 1
    else:
        Married = 2

 
    # Making predictions 
    prediction = rf_classifier.predict( 
        [[Gender, Married, ApplicantIncome, LoanAmount, Loan_Amount_Term]])
     
    if prediction == 0:
        pred = 'Rejected'
    else:
        pred = 'Approved'
    return pred
      
  
# this is the main function in which we define our webpage  
def main():       
    # front end elements of the web page 
    html_temp = """ 
    <div style ="background-color:yellow;padding:13px"> 
    <h1 style ="color:black;text-align:center;">Loan Prediction ML App</h1> 
    </div> 
    """
      
    # display the front end aspect
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # following lines create boxes in which user can enter data required to make prediction 
    Gender = st.selectbox('Gender',("Male","Female","Not Specified"))
    Married = st.selectbox('Marital Status',("Unmarried","Married","Not Specified")) 
    ApplicantIncome = st.number_input("Applicants monthly income") 
    LoanAmount = st.number_input("Total loan amount")
    Loan_Amount_Term = st.number_input("Loan Amount Term") 
    result =""
      
    # when 'Predict' is clicked, make the prediction and store it 
    if st.button("Predict"): 
        result = prediction(Gender, Married, ApplicantIncome, LoanAmount, Loan_Amount_Term ) 
        st.success('Your loan is {}'.format(result))
        print(LoanAmount)
     
if __name__=='__main__': 
    main()


Overwriting app.py


In [ ]:
!ls

app.py	rf_model.pkl  sample_data


In [ ]:
!ngrok authtoken 1mC3OgpziPyZpqpKZnvvCmXASCQ_3jjhyBCsvWNPxzRc4zzWR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!ngrok

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

In [ ]:
from pyngrok import ngrok

In [ ]:
!streamlit run app.py&>/dev/null&

In [ ]:
!pgrep streamlit

426
514
541
641
751
820
888
965
1133


In [ ]:
publ_url = ngrok.connect(port = 8501)

In [ ]:
publ_url

<NgrokTunnel: "http://2f7ea95d534b.ngrok.io" -> "http://localhost:80">

In [ ]:
#